<a href="https://colab.research.google.com/github/HwangHanJae/datascience-learningnote-pima_indians_diabetes/blob/main/pima_indians_transcription_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###[부스트코스 - 프로젝트로 배우는 데이터 사이언스](https://www.boostcourse.org/ds214)

###필요한 라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats

In [2]:
import warnings
warnings.filterwarnings('ignore')

####시각화를 선명하게 해주기 위한 설정

In [3]:
plt.style.use('ggplot')
set_matplotlib_formats('svg')

###데이터셋 로드

In [4]:
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/My CSV/diabetes_feature.csv")
df_insulin = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/DataScience/My CSV/diabetes_fill_insulin.csv")

In [6]:
df["Insulin"] = df_insulin["Insulin"]

In [7]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,148,72,35,176.45,33.6,0.627,50,1,False,False,True,False,169.5,5.138735,False
1,1,85,66,29,64.62,26.6,0.351,31,0,False,False,True,False,102.5,4.639572,True
2,8,183,64,0,186.68,23.3,0.672,32,1,True,False,True,False,169.5,5.138735,False
3,1,89,66,23,94.00,28.1,0.167,21,0,False,True,False,False,94.0,4.553877,True
4,0,137,40,35,168.00,43.1,2.288,33,1,False,False,True,False,168.0,5.129899,False


###학습과 예측에 사용할 데이터셋 만들기

In [8]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [10]:
X = df[[ 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Pregnancies_high',
      ]]
X.shape

(768, 8)

In [11]:
y = df["Outcome"]
y.shape

(768,)

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state = 42)

In [14]:
X_train.shape, y_train.shape

((614, 8), (614,))

In [15]:
X_test.shape, y_test.shape

((154, 8), (154,))

###여러개의 머신러닝 알고리즘 사용하기

In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
              RandomForestClassifier(random_state=42),
              GradientBoostingClassifier(random_state=42)]
estimators

[DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                        max_depth=None, max_features=None, max_leaf_nodes=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, presort='deprecated',
                        random_state=42, splitter='best'),
 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                        criterion='gini', max_depth=None, max_features='auto',
                        max_leaf_nodes=None, max_samples=None,
                        min_impurity_decrease=0.0, min_impurity_split=None,
                        min_samples_leaf=1, min_samples_split=2,
                        min_weight_fraction_leaf=0.0, n_estimators=100,
                        n_jobs=None, oob_score=False, random_state=42, verbose=0,
                        warm_start=False),
 GradientBoost

In [21]:
from sklearn.model_selection import RandomizedSearchCV
max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

param_distributions = {"max_depth":max_depth,
                       "max_features":max_features}
results = []
for estimator in estimators:
  result = []
  rs = RandomizedSearchCV(estimator,
                          param_distributions,
                          n_iter=100,
                          scoring="accuracy",
                          n_jobs=-1,
                          cv=5,
                          verbose=2,
                          random_state=42)
  rs.fit(X_train, y_train)
  result.append(estimator.__class__.__name__)
  result.append(rs.best_params_)
  result.append(rs.best_score_)
  result.append(rs.score(X_test,y_test))
  result.append(rs.cv_results_)
  results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done 396 tasks      | elapsed:    2.0s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.5s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:    6.7s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   26.7s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:   12.4s
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed:   46.7s
[Parallel(n_jobs=-1)]: Done 361 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  2.7min finished


In [23]:
df_result = pd.DataFrame(results)
df_result.columns = ['estimator', 'best_param','train_score','test_score','cv_result']
df_result

,estimator,best_param,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.676205009952701, 'max_depth...",0.768826,0.733766,"{'mean_fit_time': [0.007253503799438477, 0.006..."
1,RandomForestClassifier,"{'max_features': 0.5234407004789404, 'max_dept...",0.801333,0.746753,"{'mean_fit_time': [0.34291644096374513, 0.3690..."
2,GradientBoostingClassifier,"{'max_features': 0.3196385141504182, 'max_dept...",0.793229,0.720779,"{'mean_fit_time': [0.668592119216919, 0.556414..."
